In [2]:
import re

In [3]:
def remove_special_chars(text):
    remove_tag = r'。|◎|■|▼|◆|▽|・|･|\.|●|①|②|③|;|,|、|★|☆|＊|\+|◇|:|※|／／｜～|：' \
                 r'|－|-|\*|…|│|＋|←|→|\?|└|（※）|「|」|⇔|「|」|『|』|\u3000|' \
                 r'!|♪|！|~|％|%|～|\?|\／|\/|&|=|⇒|└|xx|>|<|\（|\）|\)|\(|【|】|＜|＞' \
                 r'|≪|≫|\[|\]|\r|\n|\t|\'|_|e.g|\{|\}|\#|\@|\$'
    text = re.sub(remove_tag, ' ', text)
    text = re.sub('\s\s+', ' ', text.strip())
    return text

In [7]:
def split2sentences(para):
    paras = []
    # para = remove_numbers(para)
    paras = remove_special_chars(para).split(" ")
    return paras

In [4]:
match_pattern = re.compile(r'(転職(者|社|回)*(数|で))|(次[0-9]+(社|者)目)|(転職(者|社|回)*(数|で)[0-9]+((回以上)|(社|者)*))|(数*[0-9]+((社以内)|((社|者)*経験.{0,1}[迄|(まで)|(のみ)]*)))')

In [22]:
sample_text = '【必須要件】■IT業界法人営業経験1年以上　もしくはインフラの設計もしくは運用のご経験ある方\
■IDC(データセンター)を基にした、顧客提案に興味関心をお持ちの方【歓迎要件】\
▼マネジメント経験をお持ちの方","【高学歴必須】【転職回数2回以上厳禁】【その代わり年収、超高い】'

In [14]:
def extract_changeover_age_pattern(text):
    patterns = []
    sentences = split2sentences(text)
    for sentence in sentences:
        match = re.search(match_pattern, sentence)
        if match is not None:
            patterns.append(match.string)
    return patterns

In [23]:
extract_changeover_age_pattern(sample_text)

['転職回数2回以上厳禁']